In [1]:
import os, logging, gensim
import numpy as np
import pandas as pd
from sklearn import linear_model
import unicodedata as ud
import copy

In [2]:
# Global configuration
lang = 'French'
include_pos = True
root_path = '/home/sying/Documents/Decorrelation/' + lang + '/'

if lang == 'French':
    sim_model_path = root_path + 'wolf_15k_850d.txt'
    mix_model_path = root_path + 'depglove_200d_eric.txt'
    asn_model_path = root_path + 'asn_embedding.txt'
    sig_model_path = root_path + 'sig_embedding.txt'
elif lang == 'English':
    sim_model_path = root_path + 'sim_embedding.txt'
    mix_model_path = root_path + 'glove.840B.300d.txt'
    asn_model_path = root_path + 'asn_embedding.txt'
    sig_model_path = root_path + 'sig_embedding.txt'


In [20]:
# Word Alignment in two embeddings

# file = open(sim_model_path, mode='r')
# voc, dim = file.readline().split(' ')
# voc = int(voc)
# dim_sim = int(dim)

# file_data = file.readlines()
# file.close()
# 
# sim_vocabulary = dict()
# idx = 0
# sim_model = np.zeros((voc, dim_sim), dtype=np.float32)
# word_list = []

# for line in file_data:
#     vector = line.split(' ')
#     word_list.append(vector[0])
#     sim_vocabulary[vector[0]] = idx
#     sim_model[idx,:] = vector[1:]
#     idx += 1

## POS Basic Mapping

In [109]:
sim_voc_path = '../TextFineTuning/vocabulary_pos_wolf.txt'

file = open(sim_voc_path, mode='r')
file_data = file.read()
file.close()
sim_voc_list = file_data.strip().split('\n')

In [3]:
file = open(mix_model_path, mode='r', encoding='iso-8859-1')
file_data = file.readlines()
file.close()

In [4]:
glove_raw_list = []

for line in file_data[2:]:
    vector = line.split(' ')
    glove_raw_list.append(vector[0])

In [6]:
file = open('./French_POS/vocabulary_depglove.txt', mode='w+')
file.write('\n'.join(glove_raw_list))
file.close()

In [5]:
glove_raw_list[:10]

['saint-jouin:_LOCATION_np',
 'maryland:_LOCATION_np',
 'comme_coo',
 'table_de_marbre:_ORGANIZATION_np',
 'bouqueton:_PERSON_m_np',
 'augmentation_nc',
 'araki:_PERSON_f_np',
 'gaston_febus:_PERSON_m_np',
 'jean-honoré_fragonard:_PERSON_np',
 'nicolet:_LOCATION_np']

In [149]:
glove_raw_list[7301]

'moral_adj'

In [14]:
dim_mix = int(file_data[1])
#mix_model = np.zeros((voc, dim_mix), dtype=np.float32)
mix_voc_set = set()
mix_pos_set = dict()

for vector in glove_raw_list:
    pos = vector.rsplit('_',1)[1]
    if pos in mix_pos_set:
        mix_pos_set[pos].add(vector)
    else:
        tmp = set()
        tmp.add(vector)
        mix_pos_set[pos] = tmp

In [15]:
mix_pos_set.keys()

dict_keys(['np', 'coo', 'nc', 'adj', 'v', 'ncpred', 'adv', 'pres', 'prep', 'csu', 'adjPref', 'advPref', '', 'predet', 'det', 'pro', 'poncts', 'clr', 'ponctw', 'title', 'advneg', 'pri', 'prel', 'suffAdj', 'number', 'ce', 'cln', 'aux', 'que', 'clg', 'clneg', 'N2', 'xpro', 'ilimp', 'restr', 'cld', 'cll', 'cla'])

In [19]:
glove_reject = {
    'ncpred', 'pres', 'prep', 'csu','pro','pri', 'prel',
    'adjPref', 'advPref', '', 'poncts', 'clr', 'ponctw', 'number', 'suffAdj'# Non-existant in WOLF
    'ce', 'cln', 'aux', 'que', 'clg', 'clneg', 'N2', 'xpro', 'ilimp', 'restr', 'cld', 'cll', 'cla'
}

glove_wolf_pos_mapping = {
    'np': 'n',
    'coo': 'b',
    'nc': 'n',
    'adj': 'a',
    'v': 'v',
    'adv': 'b',
    'predet': 'b',
    'det': 'b', 
    'title': 'n', 
    'advneg': 'b',
#    'ncpred': 'n'
}

In [17]:
mix_pos_set['ncpred']

{'analysé_ncpred',
 'demander_aide_ncpred',
 'compte_ncpred',
 'gué_ncpred',
 'pore_ncpred',
 "n'_avoir_rien_à_faire_ncpred",
 'faire_tache_ncpred',
 'noyé_ncpred',
 'bois_ncpred',
 'chemin_ncpred',
 'concurrence_ncpred',
 'far_ncpred',
 'dollar_ncpred',
 'commune_ncpred',
 'ut_ncpred',
 'nova_ncpred',
 'station_ncpred',
 'confiance_ncpred',
 'grâce_ncpred',
 'succès_ncpred',
 'gare_ncpred',
 'véronique_ncpred',
 'anus_ncpred',
 'bonnet_ncpred',
 'chercher_querelle_ncpred',
 'passer_maître_ncpred',
 'calme_ncpred',
 'régiment_ncpred',
 'attention_ncpred',
 'faire_pitié_ncpred',
 'joint_ncpred',
 'ne_faire_semblant_de_rien_ncpred',
 'sort_ncpred',
 'chercher_fortune_ncpred',
 'reprendre_confiance_ncpred',
 'hit_ncpred',
 'faire_alliance_ncpred',
 'fausser_compagnie_ncpred',
 'fatum_ncpred',
 'abstraction_ncpred',
 'demander_audience_ncpred',
 '_URL_ncpred',
 'joue_ncpred',
 'roux_ncpred',
 'vaincu_ncpred',
 'douce_ncpred',
 'demander_justice_ncpred',
 'chat_ncpred',
 'gagnant_ncpred',
 

In [67]:
latin_letters= {}

def is_latin(uchr):
    try: return latin_letters[uchr]
    except KeyError:
         return latin_letters.setdefault(uchr, 'LATIN' in ud.name(uchr))

def only_roman_chars(unistr):
    return all(is_latin(uchr)
           for uchr in unistr
           if uchr.isalpha()) # isalpha suggested by John Machin

In [71]:
sim_voc_roman_dict = {"grèbe_jougris|ko=큰논병아리|nl=roodhalsfuut_n": "grèbe_jougris_n", 
                      "capucin|ja=オマキザル属|lt=kapucinas|nl=kapucijnapen_n": "capucin_n", 
                      "poirier_commun|pl=grusza_pospolita|uk=груша_звичайна_n": "poirier_commun_n",
                      "gerbille|ja=スナネズミ|nl=gerbils|pl=myszoskoczki_n": "gerbille_n",
                      "nèfle|lt=šliandra|os=мугæ_n": "nèfle_n",
                      "squamates|ja=トカゲ目_n": "squamates_n",
                      "néflier_du_japon|ja=ビワ|nl=loquat|pt=nêspera_n": "néflier_du_japon_n",
                      "radioactivité_α_n": "radioactivité_α_n",
                      "α_Virginis_n": "α_Virginis_n",
                      "particule_α_n": "particule_α_n",
                      "acide_α-linolénique_n": "acide_α-linolénique_n",
                      "β-bloquant_n": "β-bloquant_n",
                      "radioactivité_β_n": "radioactivité_β_n",
                      "particule_β_n": "particule_β_n"
                     }

In [112]:
wolf_voc_set = set()
for word in sim_voc_list:
    if only_roman_chars(word):
        wolf_voc_set.add(word)
    else:
        print(word, 'not a regular form')
        if word in sim_voc_roman_dict:
            print('\tconverted to', sim_voc_roman_dict[word])
            wolf_voc_set.add(sim_voc_roman_dict[word])

ابوظبي_n not a regular form
radioactivité_α_n not a regular form
	converted to radioactivité_α_n
grèbe_jougris|ko=큰논병아리|nl=roodhalsfuut_n not a regular form
	converted to grèbe_jougris_n
गुरु_n not a regular form
Nay_نای_n not a regular form
capucin|ja=オマキザル属|lt=kapucinas|nl=kapucijnapen_n not a regular form
	converted to capucin_n
α_Virginis_n not a regular form
	converted to α_Virginis_n
particule_α_n not a regular form
	converted to particule_α_n
دبي_n not a regular form
acide_α-linolénique_n not a regular form
	converted to acide_α-linolénique_n
poirier_commun|pl=grusza_pospolita|uk=груша_звичайна_n not a regular form
	converted to poirier_commun_n
β-bloquant_n not a regular form
	converted to β-bloquant_n
radioactivité_β_n not a regular form
	converted to radioactivité_β_n
gerbille|ja=スナネズミ|nl=gerbils|pl=myszoskoczki_n not a regular form
	converted to gerbille_n
в_n not a regular form
حماس_n not a regular form
nèfle|lt=šliandra|os=мугæ_n not a regular form
	converted to nèfle_n
к_

## Define key vocabulary items

In [42]:
lpp_path = '/home/sying/Documents/TextFineTuning/lpp_tuned_wolf_vocabulary.txt'
file = open(lpp_path, mode='r')
file_data = file.read()
lpp_voc = file_data.strip().split('\n')
lpp_voc = set(lpp_voc)

In [43]:
lpp_path = '/home/sying/Documents/Similarity-Association-Benchmarks/fr-pos-wsrel.dataset'
file = open(lpp_path, mode='r')
file_data = file.read()
wsrel_voc = file_data.strip().split(';')[:2]
wsrel_voc = set(wsrel_voc)

In [44]:
lpp_path = '/home/sying/Documents/Similarity-Association-Benchmarks/fr-pos-simlex.dataset'
file = open(lpp_path, mode='r')
file_data = file.read()
simlex_voc = file_data.strip().split(';')[:2]
simlex_voc = set(simlex_voc)

In [45]:
relevant_voc = lpp_voc.union(wsrel_voc).union(simlex_voc)

In [50]:
len(relevant_voc), len(relevant_voc.difference(wolf_glove_dict.keys())), len(lpp_voc.difference(wolf_glove_dict.keys()))

(1377, 45, 45)

## Word Mapping from WOLF to GloVe
### Preparing GloVe vocabulary for multilevel search

In [124]:
def update_glove_dict(word, pos, tag, word_pos):
    old_pos = pos
    if pos in glove_wolf_pos_mapping:
        pos = glove_wolf_pos_mapping[pos]
    if word in glove_voc_dict:
        if pos in glove_voc_dict[word]:
            if word_pos in glove_voc_dict[word][pos]:
                print('Duplication at POS merging', word, pos, tag, word_pos)
            else:
                glove_voc_dict[word][pos].add(word_pos)
        else:
            tmp = set()
            tmp.add(tag)
            glove_voc_dict[word][pos] = copy.deepcopy(tmp)
    else:
        tmp = set()
        tmp.add(word_pos)
        tmp = {pos: tmp}
        glove_voc_dict[word] = copy.deepcopy(tmp)

In [150]:
glove_voc_dict = dict() # lemme -> pos -> tag

for word_pos in glove_raw_list:
    word_pos = word_pos.rstrip()
    tag = ''
    word, pos = word_pos.rsplit('_', 1)
    if ':' in word:
        word, tag = word.split(':')
    update_glove_dict(word, pos, tag, word_pos)

moral_adj
moral_nc


### Mapping

In [153]:
glove_voc_dict['moral']

{'a': {'moral_adj'}, 'n': {''}}

In [ ]:
wolf2glove_dict = dict()

for word_pos in wolf_voc_set:
    word, pos = word_pos.rsplit('_', 1)
    if word in glove_voc_dict:
        if pos in glove_voc_dict[word]:
            if len(glove_voc_dict[word][pos]) == 1:
                wolf2glove_dict[word_pos] = next(iter(glove_voc_dict[word][pos]))
            else:
                important_flag = word_pos in relevant_voc
                if important_flag:
                    candidates = list(glove_voc_dict[word][pos])
                    print("Select for ", word, 'as a', pos)
                    for idx, glove in enumerate(candidates):
                        print(idx, glove)
                    choice = -1
                    while choice not in range(len(candidates)):
                        choice = input("Make a choice: ")
                    wolf2glove_dict[word_pos] = candidates[choice]
                else:
                    
                    
                        

## Word Mapping from GloVe to WOLF

In [20]:
sim_voc_dict = dict()
for word_pos in sim_voc:
    word, pos = word_pos.rsplit('_', 1)
    if word in sim_voc_dict:
        if pos in sim_voc_dict[word]:
            print('Repeated', word_pos)
        else:
            sim_voc_dict[word].add(pos)
    else:
        tmp = set()
        tmp.add(pos)
        sim_voc_dict[word] = tmp

In [21]:
for key in list(sim_voc_dict.keys()):
    if key != key.lower():
        sim_voc_dict['+'+key.lower()] = key

In [22]:
# Support data structure
glove_wolf_dict = dict()
wolf_glove_dict = dict()
no_hit_word_dict = dict()
no_hit_pos_set = set()
dup_set = []
no_hit_dup_set = []

for line in file_data[2:]:
    vector = line.split(' ')
    word_ori = vector[0]
    word, pos = word_ori.rsplit('_',1)
    if pos not in glove_wolf_pos_mapping:
        continue
    else:
        pos = glove_wolf_pos_mapping[pos]
    if ':' in word:
        word = word.split(':')[0]
    if word not in sim_voc_dict and '+'+word not in sim_voc_dict:
        unmatched_word = word+'_'+pos
        if unmatched_word in no_hit_word_dict:
                no_hit_dup_set.append((unmatched_word, no_hit_word_dict[unmatched_word], vector[0]))
                print('Duplicate', unmatched_word, no_hit_word_dict[unmatched_word], vector[0])
        else:
            no_hit_word_dict[unmatched_word] = vector[0]
    else:
        if word in sim_voc_dict:
            pass
        elif  '+'+word in sim_voc_dict:
            word = sim_voc_dict['+'+word]
        else:
            print("Control flow error.")
        matched_word = word+'_'+pos
        if pos not in sim_voc_dict[word]:
            no_hit_pos_set.add(matched_word)
        else:
            glove_wolf_dict[vector[0]] = matched_word
            if matched_word in wolf_glove_dict:
                dup_set.append((matched_word, wolf_glove_dict[matched_word], vector[0]))
                print('Duplicate matching', matched_word, wolf_glove_dict[matched_word], vector[0])
            else:
                wolf_glove_dict[matched_word] = vector[0]

Duplicate sainjon_n sainjon:_PERSON_m_np sainjon:_PERSON_np
Duplicate filiale_n filiale_nc filiale:_COMPANY_np
Duplicate matching Garnier_n garnier:_PERSON_np garnier:_PERSON_f_np
Duplicate bull_red_n bull_red:_PERSON_m_np bull_red:_COMPANY_np
Duplicate dupuy_n dupuy:_LOCATION_np dupuy:_PERSON_np
Duplicate hoffmann_n hoffmann:_PERSON_np hoffmann:_PERSON_m_np
Duplicate parks_n parks:_PERSON_np parks:_PERSON_m_np
Duplicate parker_n parker:_LOCATION_np parker:_PERSON_f_np
Duplicate matching Murray_n Murray_np murray:_LOCATION_np
Duplicate toto_n toto:_COMPANY_np toto_nc
Duplicate big_horn_n big_horn:_PERSON_np big_horn:_LOCATION_np
Duplicate matching salle_n salle:_PERSON_m_np salle_nc
Duplicate carol_n carol:_PERSON_f_np carol:_PERSON_m_np
Duplicate matching casino_n casino:_COMPANY_np casino:_LOCATION_np
Duplicate tas_n tas_nc tas:_COMPANY_np
Duplicate matching Alfred_n Alfred_np alfred:_PERSON_m_np
Duplicate matching Daniel_n Daniel_np daniel:_PERSON_np
Duplicate ewing_n ewing:_PERSON_

In [49]:
for word in relevant_voc.difference(wolf_glove_dict.keys()):
    print('"'+word+'",')

"que_b",
"vingt-six_n",
"orgueilleuse_a",
"s'enfuir_v",
"21_n",
"8_n",
"10_n",
"serré_a",
"où_b",
"croître_b",
"salé_a",
"vingt-deux_n",
"3_n",
"mille_a",
"amérique_n",
"Pacifique_n",
"bon_après-midi_n",
"14_n",
"universel_n",
"s'occuper_de_v",
"ne_b",
"2_n",
"s'endormir_v",
"4_n",
"13_n",
"12_n",
"doux_n",
"b_n",
"19_n",
"ta_n",
"se_réveiller_v",
"vas_n",
"se_taire_v",
"voici_b",
"chef-d'œuvre_n",
"tu_n",
"chapitre_b",
"riposter_v",
"attendre_avec_impatience_v",
"quand_b",
"remords_n",
"œil_n",
"rouillé_a",
"se_hâter_v",
"23_n",


In [ ]:
false_vocabulary = {"que_b", "vingt-six_n", "21_n", "8_n", "10_n", "où_b", "vingt-deux_n", "3_n", "14_n", "ne_b", "2_n", "4_n", "13_n", "12_n", "b_n", "19_n",
                    "ta_n", "23_n", "tu_n", }

to_manual_match = {
    "orgueilleuse_a", "s'enfuir_v", "serré_a", "croître_b", "salé_a", "mille_a", "amérique_n", "Pacifique_n", "bon_après-midi_n", "universel_n",
    "s'occuper_de_v", "s'endormir_v", "doux_n", "se_réveiller_v", "vas_n", "se_taire_v", "voici_b", "chef-d'œuvre_n",
    "chapitre_b", "riposter_v", "attendre_avec_impatience_v", "quand_b", "remords_n", "œil_n", "rouillé_a", "se_hâter_v"}

### Solve Matched Duplicates

In [1]:
file = open('./French_POS/Duplicates.txt', mode='r')
file_data = file.readlines()
file.close()

matched_dup = dict()

for line in file_data:
    data = line.strip().split(' ')
    if len(data) == 5:
        if data[2] in matched_dup:
            matched_dup[data[2]].add(data[3])
            matched_dup[data[2]].add(data[4])
        else:
            tmp = set()
            tmp.add(data[3])
            tmp.add(data[4])            
            matched_dup[data[2]] = tmp

In [2]:
for key, value in matched_dup.items():
    candidates = list(value)
    print("Select for ", key)
    for idx, glove in enumerate(candidates):
        print(idx, glove)
    choice = input("Make a choice: ")
    

{'égard_n': {'égard_nc', 'égard_ncpred'},
 'poil_n': {'poil:_LOCATION_np', 'poil_nc', 'poil_ncpred'},
 'salle_n': {'salle:_LOCATION_np', 'salle:_PERSON_m_np', 'salle_nc'},
 'cours_n': {'cours:_LOCATION_np', 'cours_nc', 'cours_ncpred'},
 'casino_n': {'casino:_COMPANY_np', 'casino:_LOCATION_np', 'casino_nc'},
 'physique_n': {'physique:_LOCATION_np', 'physique_nc'},
 'allemand_n': {'allemand:_LOCATION_np',
  'allemand:_PERSON_m_np',
  'allemand_nc',
  'allemand_ncpred'},
 'ferry_n': {'ferry:_PERSON_f_np',
  'ferry:_PERSON_m_np',
  'ferry:_PERSON_np',
  'ferry_nc'},
 'thriller_n': {'thriller:_ORGANIZATION_np', 'thriller_nc'},
 'logarithme_n': {'logarithme_nc', 'logarithme_ncpred'},
 'habitat_n': {'habitat:_COMPANY_np', 'habitat_nc'},
 'gouverneur_n': {'gouverneur:_LOCATION_np',
  'gouverneur_nc',
  'gouverneur_ncpred'},
 'carte_n': {'carte:_PERSON_m_np', 'carte_nc', 'carte_ncpred'},
 'forêt_n': {'forêt:_LOCATION_np', 'forêt:_PERSON_m_np', 'forêt_nc'},
 'stein_n': {'stein:_LOCATION_np', 'st

In [23]:
dupped_wolf = set([w1 for w1, _, _ in dup_set])

In [24]:
len(no_hit_word_dict), len(no_hit_pos_set)

(122058, 3025)

In [25]:
len(wolf_glove_dict), len(dup_set), len(dupped_wolf)

(26290, 3239, 2081)

In [30]:
len(lpp_voc)

1375

In [41]:
wolf_glove_dict['Amour_n']

'Amour_np'

In [29]:
lpp_voc.difference(glove_wolf_dict.values())

{'10_n',
 '12_n',
 '13_n',
 '14_n',
 '19_n',
 '21_n',
 '23_n',
 '2_n',
 '3_n',
 '4_n',
 '8_n',
 'Pacifique_n',
 'amérique_n',
 'attendre_avec_impatience_v',
 'b_n',
 'bon_après-midi_n',
 'chapitre_b',
 "chef-d'œuvre_n",
 'croître_b',
 'doux_n',
 'mille_a',
 'ne_b',
 'orgueilleuse_a',
 'où_b',
 'quand_b',
 'que_b',
 'remords_n',
 'riposter_v',
 'rouillé_a',
 "s'endormir_v",
 "s'enfuir_v",
 "s'occuper_de_v",
 'salé_a',
 'se_hâter_v',
 'se_réveiller_v',
 'se_taire_v',
 'serré_a',
 'ta_n',
 'tu_n',
 'universel_n',
 'vas_n',
 'vingt-deux_n',
 'vingt-six_n',
 'voici_b',
 'œil_n'}

In [37]:
len(relevant_voc)

2607

### Solved Unmatched Duplicates for Benchmarks

In [38]:
dupped_unmatch = set([w1 for w1, _, _ in no_hit_dup_set])

In [43]:
len(no_hit_dup_set), len(dupped_unmatch), len(dupped_unmatch.intersection(relevant_voc))

(7489, 5585, 0)

In [ ]:
vocabulary_mask = np.ones(voc, dtype=np.bool)
vocabulary_mask[list(vocabulary.values())] = 0

sim_model_red = sim_model[vocabulary_mask,:]
mix_model_red = mix_model[vocabulary_mask,:]

In [23]:
# Linear Decorrelation by projecting A emb. onto B emb.

reg = linear_model.LinearRegression()
reg.fit(sim_model_red, mix_model_red)
score = reg.score(sim_model_red, mix_model_red)
print("Model mapping score: ", score)

asn_model_red = mix_model_red - reg.predict(sim_model_red)
word_list = [word_list[i] for i in range(len(word_list)) if vocabulary_mask[i]]
file = open(asn_model_path, mode='w')
file.write('{} {}\n'.format(len(word_list), dim_mix))
asn_model_df = pd.DataFrame(asn_model_red, index=word_list)
file.write(asn_model_df.to_csv(sep=' ', header=False))
file.close()

file = open(sig_model_path, mode='w')
file.write('{} {}\n'.format(len(word_list), dim_mix))
sig_model_red = reg.predict(sim_model_red)
sig_model_df = pd.DataFrame(sig_model_red, index=word_list)
file.write(sig_model_df.to_csv(sep=' ', header=False))
file.close()


Model mapping score:  0.9551728942373527


In [24]:
sig_model_df.head()

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
1,0.012603,0.020706,0.028368,0.058304,-0.055454,-0.021133,0.084205,0.022269,0.048857,-0.013411,...,-0.028009,-0.003340,-0.030710,-0.011457,0.057529,0.019761,-0.041603,0.103786,0.055567,-0.073017
5,0.019748,-0.012024,-0.086954,-0.072426,-0.041885,-0.034332,-0.057955,-0.024164,-0.074774,-0.046486,...,0.091845,0.047060,-0.059097,-0.062586,-0.030236,-0.033847,0.024008,-0.043434,0.067957,0.088192
abbé,-0.010245,-0.084457,-0.021803,-0.057587,0.034122,0.103947,0.142134,-0.016178,-0.113889,-0.001802,...,0.029358,0.047447,-0.043896,-0.010759,-0.083473,0.005749,0.027510,0.042458,-0.109894,-0.058533
abc,0.117906,-0.068092,-0.000950,0.047642,0.135681,0.014042,0.137647,-0.004201,-0.094872,-0.047190,...,-0.061683,-0.020689,-0.039703,0.161352,-0.117802,-0.042281,-0.035583,0.041450,-0.076077,-0.033600
abondance,-0.008791,0.072128,0.024740,-0.014126,0.033405,-0.155098,0.001989,-0.106936,0.143875,-0.082115,...,-0.047378,-0.008244,0.152733,0.008324,0.022903,-0.127469,0.049498,-0.001816,0.021872,0.191925


In [25]:
asn_sig_corr = asn_model_df.corrwith(sig_model_df, method='spearman', axis=0) 

In [26]:
len(word_list)

635

In [27]:
sorted(asn_sig_corr.abs())

[0.0010311066640148138,
 0.0024577418399294253,
 0.010510271710257962,
 0.011380298550146042,
 0.013384467804274726,
 0.02465601298404169,
 0.029727889972051027,
 0.03630448292854364,
 0.03714847437658877,
 0.04249854831475629,
 0.0447806376343271,
 0.04554601206867216,
 0.046640410651423936,
 0.047483652401216894,
 0.048106768496636865,
 0.05033626666317044,
 0.05096537893392079,
 0.05310864635233671,
 0.05542736045554852,
 0.05546832164884837,
 0.05619859383361809,
 0.05709214774735412,
 0.057927916124784345,
 0.05887679440138292,
 0.06066514345550571,
 0.06390623298714763,
 0.06642640006023265,
 0.06802709610961355,
 0.06946378413761208,
 0.06975267078462578,
 0.07291176595955731,
 0.07312163177388671,
 0.07409600839507606,
 0.07449516912180354,
 0.07462975372350052,
 0.07734370876310037,
 0.0774859995545358,
 0.07802694534024267,
 0.07855543559591822,
 0.07877442139824023,
 0.07995242971347466,
 0.07996196607960979,
 0.08017935941729587,
 0.08292705965642726,
 0.083057463338298,
 0

In [3]:
# Extract vocabulary
file = open(sim_model_path, mode='r')
voc, dim = file.readline().split(' ')
voc = int(voc)
dim_sim = int(dim)


file_data = file.readlines()
file.close()

vocabulary = dict()
idx = 0
word_list = []

for line in file_data:
    vector = line.split(' ')
    word_list.append(vector[0])
    vocabulary[vector[0]] = idx


In [6]:
voc_file_path = root_path + 'sim_vocabulary.txt'
file = open(voc_file_path, mode='w')
file.write('\n'.join(list(vocabulary.keys())))
file.close()
     

In [7]:
file = open(mix_model_path, mode='r', encoding="cp1252")
file_data = file.readlines()
file.close()

voc_file_path = root_path + 'mix_vocabulary.txt'
file = open(voc_file_path, mode='w')


for line in file_data[2:]:
    vector = line.split(' ')
    word = vector[0].split(':')[0]
    file.write(word+'\n')
    
file.close()


In [75]:
alphabet_wolf = set()
for word_pos in sim_voc:
    word, _ = word_pos.rsplit('_', 1)
    alphabet_wolf.update(word)
    
alphabet_glove = set()
for word_pos in glove_raw_list:
    word, _ = word_pos.rsplit('_',1)
    if ':' in word:
        word = word.split(':')[0]
    alphabet_glove.update(word)

In [76]:
''.join(list(alphabet_glove.union(alphabet_wolf)))

'ṃ;ÈñZamĀHī]ứr(BzČÉİÿîö¼ŒủyŌj:9à8Åi´%ÜJVNã™-põ+ė1/`,ÐµT#ù!6ā\x9cú\\AÖṣ¯fëÂGX»ÎâEFKw°żgheSệćō^Otọ[ø×üûł‘<\'åqĲváÄP¬MUQşœê«C7ả|șšṇçń?Đ’Çl\xa0năd_Læ"céDśèęþuř̐xï∨)YRÆ&äčąó.ūž>Wßĥ0kí=ḷ4ň3òìýÃ2sô5oIṛ‑½bÁ'

In [77]:

text = 'ṃ;È큰ñгZαamĀHビī]иứrβي(BzČÉİÿیîö¼カрŒủyगŌj:кو9オà8Åi´%ÜJVحNã™ズ-põ+ė1/`ن,вÐзµT#ù!6ā\x9cú리\\AÖṣ¯а병йfëÂGX»ÎâEFKw°żgheSệćōネ^Otọн[øد×üûł‘<\'ワåqĲvبظáÄчP¬MUुQşœê«ゲナザキC7уả|ș아šṇçń?रĐ’マスÇ논l\xa0مnăd_Læ"céDśèęþuř̐xï∨)YRÆ&äčミąó.ūžا>سWßмル0ĥkí=属ḷ4ň3òшìýÃ2sô5oIṛ‑½b目Áト'
text = unicodedata.normalize('NFD', text)
text = text.encode('ascii', 'ignore')
text = text.decode("utf-8")

In [78]:
text

'm;EnZamAHi]ur(BzCEIyiouyOj:9a8Ai%UJVNa-po+e1/`,T#u!6au\\AOsfeAGXIaEFKwzgheSeco^Oto[uu<\'aqvaAPMUQseC7a|ssncn?Clnad_L"ceDseeurxi)YR&acao.uz>W0hki=l4n3oiyA2so5oIrbA'

In [64]:
'ṃ;ÈñZamĀHī]ứr(BzČÉİÿîö¼ŒủyŌj:9à8Åi´%ÜJVNã™-põ+ė1/`,ÐµT#ù!6ā\x9cú\\AÖṣ¯fëÂGX»ÎâEFKw°żgheSệćō^Otọ[ø×üûł‘<\'åqĲváÄP¬MUQşœê«C7ả|șšṇçń?Đ’Çl\xa0năd_Læ"céDśèęþuř̐xï∨)YRÆ&äčąó.ūž>Wßĥ0kí=ḷ4ň3òìýÃ2sô5oIṛ‑½bÁ'
'm;EnZamAHi]ur(BzCEIyio  uyOj:9a8Ai %UJVNa -po+e1/`,  T#u!6au    \\AOs feAGX IaEFKw zgheSeco^Oto[  uu  <\'aq vaAP MUQs e C7a|ssncn?  Cl    nad_L "ceDseeurxi)YR&acao.uz>W0hki=l4n3oiyA2so5oIrbA'

'm;EnZamAHi]ur(BzCEIyiouyOj:9a8Ai%UJVNa-po+e1/`,T#u!6au\\AOsfeAGXIaEFKwzgheSeco^Oto[uu<\'aqvaAPMUQseC7a|ssncn?Clnad_L"ceDseeurxi)YR&acao.uz>W0hki=l4n3oiyA2so5oIrbA'

In [90]:
normalized_alphabet = {':', '_', '-', "'", 'é'}
import string
normalized_alphabet.update(string.ascii_lowercase)

In [92]:
glove_raw_list[0]

'saint-jouin:_LOCATION_np'

In [91]:
for word in glove_raw_list:
    word = word.rsplit('_', 1)[0]
    if ':' in word:
        word = word.split(':')[0]
    if set(word).difference(normalized_alphabet) != set():
        print(word)

goûts
Éthiopie
 socialisation
Tommaso
Lyon
sainte_geneviève
Evocation
orcières
avant-scène
château-d'eau
Hudson
perrière
cantilène
anthropoïde
stoïcien
Minnie
maximilien_de_bavière
Bertha
alliage-mère
saint-françois
d'esch-sur-sûre
Rain
dépêtrer
Luiz
déboîter
Eliot
césène
 faiseur
fédération_française_de_spéléologie
châtelneuf
port-jérôme
belvédère
Wendell
Annunziata
Maris
Vouel
Sinogramme
Stephen
Sadie
Wynn
eugène_de_savoie
kolozsvár
Charles
Myriam
la_poste_française
azerbaïjan
brûlé
Praise
harald_ier_de_norvège
Síle
adélaïde
Émile_zola
sean_s._cunningham
nicolas_sarkozy 
würm
Florian
chérel_mickaël
Hanna
route_a40
Ivan
jacques_debû-bridel
benoît_thieulin
hermann_sörgel
Victorius
Carla
Cadell
gibecière
croix-rouge_française
Édouard_andré
Lyubov
Antoinette
côte
Manya
Bologne
mamère
agnès_de_méranie
 type
Wesley
procèsverbal
Thiron-Gardais
jaromír_jágr
franz_müntefering
pierre-georges_latécoère
schizophrène
david_r._ellis
Phoenix
mère
glyptothèque
château_de_vaux-le-vicomte
Lore
haïm_we

In [ ]:
def normalize(string):
    